<a href="https://colab.research.google.com/github/kkrusere/Developing-a-Score-to-Measure-Riskiness-of-Residential-Properties-Insurance/blob/main/data_collection_prep_and_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


## <center> **Developing a Score to Measure Riskiness of Residential Properties Insurance** 

<center><em>Developing a Score to Measure Riskiness of Residential buildings, Homes, Apartments and Condos as part of an insurance policy underwriting. Insurance underwriting is how an insurance company evaluates its risk. In this project, we identify and explore multiple data sources to collect variables that could be used to develop a score that measures the riskiness of residential buildings to aid the insurance underwriting process.</em></center>

<center><img src="https://github.com/kkrusere/Developing-a-Score-to-Measure-Riskiness-of-Residential-Properties-Insurance/blob/main/assets/real-estate-risk.jpg?raw=1" width=600/></center>

***Project Contributors:*** Kuzi Rusere and Umair Shaikh<br>
**MVP streamlit App URL:** https://kkrusere-developing-a-score-to-measure-prototype-mvp-app-acxav4.streamlitapp.com





### **Data collection**

This notebook is for the data collection, cleaning and preparation. The first dataset that we are going to be using is from the New York City OpenDataset.

<center><img src="https://github.com/kkrusere/Developing-a-Score-to-Measure-Riskiness-of-Residential-Properties-Insurance/blob/main/assets/nycOpenData.png?raw=1" width=600/></center>

The NYC OpenData is a data registry/repositoory of public data generated by various New York City agencies and other City organizations that is publicly available and accessible for anyone to use, participate in and improve government by conducting research and analysis gaining a better understanding of the services provided by City. The repository is an initiative to improve the accessibility, transparency, and accountability of City government.


The datasets are available and accessible in a variety of machine-readable formats including API access. We are going to be using the NYC 311 dataset. NYC 311 gives access to non-emergency City services and info about City government programs, this tool was launched in 2003 with phone call as only contact Type, but now is also accessible through text messages, chat, a mobile application, social media and a website.
<br>
<br>
<br>
<center><img src="https://github.com/kkrusere/Developing-a-Score-to-Measure-Riskiness-of-Residential-Properties-Insurance/blob/main/assets/311_contact_type.png?raw=1" width=600/><figcaption><em>Image from: https://council.nyc.gov/data/311-services/</em></figcaption></center>

<br>
<br>
<br>

We are going to be using 311 data with `Request Types` related to incidents,complaints tied to residential areas. So, right from the bet, our data in the `Location Type` will be filtered to only include entries with `Residential` as you will see from the below.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


The data that we are woking with is huge, so we are going to be using pyspark for reading the dataframe/table and then use pandas to filter and clean the data.
## **Setting up pyspark:**
Installing PySpark on Google Colab is to use pip install

In [3]:
# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 76.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a679c9e2ca1f628ef73a5b8284ac8300a7ae32262bf4f624a601b2daa3e8acf7
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
# # Import SparkSession
# from pyspark.sql import SparkSession
# # Create a Spark Session
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# # Check Spark Session Information
# spark

We will use spark to read the csv dataframe 

In [5]:
#data_df = spark.read.csv("/content/drive/MyDrive/capstone/311_Service_Requests_from_2010_to_Present.csv", header=True, inferSchema=True)
data_df = pd.read_csv("/content/drive/MyDrive/capstone/311_Service_Requests_from_2010_to_Present.csv")

In [6]:
data_df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,25595691,05/23/2013 12:00:00 AM,05/29/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,11209.0,7207 3 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.633380,-74.026993,"(40.63338019237986, -74.02699256583904)"
1,25595692,05/23/2013 12:00:00 AM,06/03/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10457.0,269 EAST BURNSIDE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.851333,-73.902133,"(40.851332558936704, -73.90213313592302)"
2,25595877,05/23/2013 12:20:25 PM,09/06/2013 12:00:00 AM,DSNY,Department of Sanitation,Graffiti,Graffiti,Residential,10472.0,1963 HAVILAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.829475,-73.858298,"(40.829474814637784, -73.85829772136906)"
3,25595984,05/23/2013 12:00:00 AM,05/31/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,11229.0,1820 AVENUE V,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.597049,-73.952872,"(40.59704910449971, -73.95287153097844)"
4,25596010,05/23/2013 12:00:00 AM,06/08/2013 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10467.0,3535 ROCHAMBEAU AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.882408,-73.879058,"(40.88240811038497, -73.87905847713522)"


In [7]:
#lets take a look atn the columns that we have in our dataset
list(data_df.columns)

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location']

In [8]:
#we remove the columns/features of our dataset that are not going to be of any use for this project 
data_df = data_df[['Unique Key','Created Date','Agency','Agency Name','Complaint Type','Descriptor','Location Type','Incident Zip','Incident Address','Street Name','Address Type','City','Resolution Description','Borough','Latitude','Longitude',]]

In [9]:
list(data_df.columns)

['Unique Key',
 'Created Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Address Type',
 'City',
 'Resolution Description',
 'Borough',
 'Latitude',
 'Longitude']

Data cleaning and preparation

In [10]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6289602 entries, 0 to 6289601
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unique Key              int64  
 1   Created Date            object 
 2   Agency                  object 
 3   Agency Name             object 
 4   Complaint Type          object 
 5   Descriptor              object 
 6   Location Type           object 
 7   Incident Zip            float64
 8   Incident Address        object 
 9   Street Name             object 
 10  Address Type            object 
 11  City                    object 
 12  Resolution Description  object 
 13  Borough                 object 
 14  Latitude                float64
 15  Longitude               float64
dtypes: float64(3), int64(1), object(12)
memory usage: 767.8+ MB


In [11]:
#we are going to drop nan from our dataset, we have plenty enough data that we can afford to drop rows
data_df.dropna(axis = 0, how ='any', inplace=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6048735 entries, 0 to 6289601
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unique Key              int64  
 1   Created Date            object 
 2   Agency                  object 
 3   Agency Name             object 
 4   Complaint Type          object 
 5   Descriptor              object 
 6   Location Type           object 
 7   Incident Zip            float64
 8   Incident Address        object 
 9   Street Name             object 
 10  Address Type            object 
 11  City                    object 
 12  Resolution Description  object 
 13  Borough                 object 
 14  Latitude                float64
 15  Longitude               float64
dtypes: float64(3), int64(1), object(12)
memory usage: 784.5+ MB


In [ ]:
#now we change datatypes for the `Unique Key`, `Created Date`, and `Incident Zip`
data_df['Unique Key'] = data_df['Unique Key'].astype(object)
data_df['Incident Zip'] = data_df['Incident Zip'].astype(str)
#we are going to split the `Created Date` into date and time
data_df['Date'] = [ele.split(" ")[0] for ele in data_df['Created Date']]
data_df['Time'] = [f"{ele.split(' ')[1]} {ele.split(' ')[2]}" for ele in data_df['Created Date']]
#we drop the the `Created Date` column
data_df.drop('Created Date', axis=1, inplace=True)
#now we change datatypes of the `Date` and `Time` to the datetime format 
data_df['Date'] = pd.to_datetime(data_df['Date'], format='%m/%d/%Y')
data_df['Time'] = pd.to_datetime(data_df['Time'], format='%H:%M').dt.time

In [ ]:
data_df.info()

In [ ]:
data_df.head()

In [17]:
#we install the Python SQL Toolkit and Object Relational Mapper and the python MySQL connector
!pip install SQLAlchemy
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 123.9 MB/s 


In [18]:
#change directory so that we can access the config.py file 
%cd /content/drive/MyDrive/capstone

/content/drive/MyDrive/capstone


Storing our cleaned data set to a MySQL AWS RDS

In [ ]:
import mysql.connector as connection
from sqlalchemy import create_engine
import config #this holds our credentials for the database 

host= config.host
user= config.user
db_password = config.password
port = config.port

#create the connection to the AWS MySQL database
conn = connection.connect(
  host=host,
  user=user,
  password=db_password,
  port = port,
)
mycursor = conn.cursor()

In [ ]:
#we create the database to store our 311 dataset
mycursor.execute("CREATE DATABASE IF NOT EXISTS NYC311_db")
database = "NYC311_db"


In [ ]:
# create sqlalchemy engine and converting our pandas dataframe to an SQL table
engine = create_engine(f"mysql+pymysql://{user}:{db_password}@{host}/{database}")
data_df.to_sql('NYC311Open_Data', con = engine, if_exists = 'append', index=False)